EXAM johanna GALVIS RODRIGUEZ

In [170]:
import numpy as np
np.set_printoptions(threshold=10000,suppress=True)
import pandas as pd
import warnings
import matplotlib.pyplot as plt
from sklearn import model_selection
warnings.filterwarnings('ignore')
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import tree
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score, KFold, cross_val_predict
import time

In [171]:
from sklearn.impute import SimpleImputer as Imputer
from sklearn.preprocessing import OneHotEncoder

In [172]:
datatrain = pd.read_csv("Train.csv", sep=';', header=0, index_col=0, decimal=",")
datatrain.head(2) 

,LB,AC,FM,UC,ASTV,MSTV,ALTV,MLTV,DL,DS,...,Min,Max,Nmax,Nzeros,Mode,Mean,Median,Variance,Tendency,Classe
ID,,,,,,,,,,,,,,,,,,,,,
1,120,0,0,0,73,0.5,43,2.4,0,0,...,62,126,2,0,120,137,121,73,1,2
2,132,4,0,4,17,2.1,0,10.4,2,0,...,68,198,6,1,141,136,140,12,0,1


In [173]:
datatest = pd.read_csv("Test.csv", sep=';', header=0, index_col=0, decimal=',')
datatest.head(2) 

,LB,AC,FM,UC,ASTV,MSTV,ALTV,MLTV,DL,DS,...,Width,Min,Max,Nmax,Nzeros,Mode,Mean,Median,Variance,Tendency
ID,,,,,,,,,,,,,,,,,,,,,
1639,130,2,0,4,25,1.7,0,0.0,4,0,...,94,76,170,6,0,133,119,121,73,0
1640,133,1,0,4,25,1.7,0,0.0,4,0,...,94,76,170,6,0,133,117,117,77,0


In [174]:
datatrain.shape

(1638, 23)

In [175]:
preX = datatrain.values
preX = preX.astype(float)
X_train = np.copy(preX[:,0:22]) 


In [176]:
Y_train = np.copy(preX[:,22:23]) 

In [177]:
# format target variable:
Y_train[Y_train == 1] = 0
Y_train[Y_train == 2] = 1

In [178]:
X_test = datatest.values
X_test = X_test.astype(float)
X_test.shape, X_train.shape

((488, 22), (1638, 22))

In [179]:
scaler = StandardScaler()
Xnorm_train = scaler.fit_transform(X_train)

In [180]:
Xnorm_test = scaler.transform(X_test)

In [181]:
# EFFICIENT PARAMETER TUNING :
parRF ={
    'n_estimators' : [50,70]
}
grid_RF = model_selection.GridSearchCV(RandomForestClassifier(), parRF, n_jobs=4, scoring='accuracy')

parknn = {  
   'n_neighbors': [5,7,11,13,15],
    'weights' : ['uniform', 'distance']
} 

grid_knn = model_selection.GridSearchCV(KNeighborsClassifier(), parknn, cv=5,
                                      scoring='accuracy')

parmlp = {
    'hidden_layer_sizes': [(40,20), (45,23) , (50,30)],
    'activation' : ['tanh', 'relu'],
    'alpha' : [1e-3, 1e-4, 1e-5],
    'solver' : ['lbfgs', 'sgd', 'adam'],
    'max_iter' : [100,200]
}
grid_mlp = model_selection.GridSearchCV(MLPClassifier(), parmlp, cv=5, n_jobs=4,
                             scoring='accuracy') 

parbag ={
    'n_estimators' : [50,70]
}
grid_bag = model_selection.GridSearchCV(BaggingClassifier(), parbag, n_jobs=4, scoring='accuracy')

In [182]:
print("searching best parameters(gridSearcCV)")

print("RandomForestClassifier")
grid_RF.fit(Xnorm_train, Y_train)
print(grid_RF.best_params_)
print(grid_RF.best_score_)

print("KNN")
grid_knn.fit(Xnorm_train, Y_train)
print(grid_knn.best_params_)
print(grid_knn.best_score_)

print("MLP")
grid_mlp.fit(Xnorm_train, Y_train)
print(grid_mlp.best_params_)
print(grid_mlp.best_score_)



searching best parameters(gridSearcCV)
RandomForestClassifier
{'n_estimators': 50}
0.9316252703811443
KNN
{'n_neighbors': 7, 'weights': 'uniform'}
0.8736257179085551
MLP
{'activation': 'tanh', 'alpha': 0.001, 'hidden_layer_sizes': (45, 23), 'max_iter': 200, 'solver': 'adam'}
0.9243063325128664


In [183]:
print("bagging")
grid_bag.fit(Xnorm_train, Y_train)
print(grid_bag.best_params_)
print(grid_bag.best_score_)


bagging
{'n_estimators': 70}
0.9199951517863804


In [184]:
# dans mes résultats, MLP meme avec paramètres optimisés n'etait pas en tête 
# pour les accuracy (et prennais enormement de temps )
# je teste plusieurs méthodes sauf MLP:

In [185]:
# je mets à jour les params des algos que j'ai pu optimiser (pas le temps pour optim tous avec grid pour cet exam)
clfs = {
    'RF': RandomForestClassifier(n_estimators=70, random_state=1),
    'KNN2' : KNeighborsClassifier(n_neighbors=7, weights='uniform'),
    'ADA' : AdaBoostClassifier(n_estimators=50, random_state=1),
    'BAG' : BaggingClassifier(n_estimators=50) ,
    'NB' : GaussianNB(),
    'CART' : DecisionTreeClassifier(criterion='gini'),
    'ID3' : DecisionTreeClassifier(criterion='entropy'),
    'ST' : DecisionTreeClassifier(max_depth=1) #decisionStump
} 

In [186]:
## applying KFold method to do re-sampling (cross validation) when running
def run_classifiers(clfs, X, Y):
    dico = {'classifier':[],'accuracy_mean':[],'accuracy_sd':[], 
             'precision_mean':[], 'precision_sd':[], 'recall_mean' : [], 'recall_sd' : [],
            'AUC':[], 'time_s':[]} #output into dictionnary
    kf = KFold(n_splits=5, shuffle=True, random_state=0)
    for clf_id in clfs:
        initime = time.time()
        clf = clfs[clf_id]
        cvAccur = cross_val_score(clf, X, Y, cv=kf, scoring='accuracy')
        end = time.time()
        cvPrecision = cross_val_score(clf, X, Y, cv=kf, scoring='precision')
        cvRecall = cross_val_score(clf, X, Y, cv=kf, scoring='recall')
        cvAUC = cross_val_score(clf, X, Y, cv=kf, scoring='roc_auc')
        dico['classifier'].append(clf_id)
        dico['accuracy_mean'].append(round(np.mean(cvAccur),3))
        dico['accuracy_sd'].append(round(np.std(cvAccur),3))
        dico['precision_mean'].append(round(np.mean(cvPrecision),3))
        dico['precision_sd'].append(round(np.std(cvPrecision),3))
        dico['recall_mean'].append(round(np.mean(cvRecall),3))
        dico['recall_sd'].append(round(np.std(cvRecall),3))
        dico['AUC'].append(round(np.mean(cvAUC),3))
        dico['time_s'].append(round((end-initime),3))
    return dico

In [187]:
dd = run_classifiers(clfs, X_train, Y_train)
pd.DataFrame.from_dict(dd)

,classifier,accuracy_mean,accuracy_sd,precision_mean,precision_sd,recall_mean,recall_sd,AUC,time_s
0,RF,0.954,0.014,0.947,0.009,0.880,0.050,0.989,0.896
1,KNN2,0.916,0.009,0.905,0.013,0.770,0.029,0.956,0.117
2,ADA,0.952,0.009,0.924,0.016,0.900,0.034,0.987,0.705
3,BAG,0.957,0.015,0.959,0.013,0.891,0.036,0.986,1.727
4,NB,0.897,0.024,0.826,0.035,0.784,0.095,0.957,0.011
5,CART,0.946,0.016,0.896,0.011,0.897,0.042,0.931,0.049
6,ID3,0.935,0.015,0.876,0.048,0.883,0.044,0.922,0.037
7,ST,0.926,0.007,0.947,0.017,0.768,0.027,0.876,0.012


In [188]:
# le meilleur étant le bagging, je le choisis pour la prédiction

In [189]:
# HERE I DO PREDICTION
meth = BaggingClassifier(n_estimators=50, random_state=1)
meth.fit(Xnorm_train, Y_train)
predicted = meth.predict(Xnorm_test)

In [194]:
len(predicted)

488

In [191]:
individus = datatest.index.values

In [192]:
individus = datatest.index.values
dfout = pd.DataFrame(predicted, index=individus, columns=["predicted"])
dfout

,predicted
1639,0.0
1640,0.0
1641,0.0
1642,1.0
1643,0.0
...,...
2122,1.0
2123,1.0
2124,1.0
2125,1.0


In [193]:
dfout.to_csv("prediction_galvis_johanna.csv", sep=",")